In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
transfers_csv = "../Resources/transfers.csv"
transfers_df = pd.read_csv(transfers_csv)
transfers_df.head()

,Name,Position,Age,Team_from,League_from,Team_to,League_to,Season,Market_value,Transfer_fee
0,Luís Figo,Right Winger,27,FC Barcelona,LaLiga,Real Madrid,LaLiga,2000-2001,NaN,60000000
1,Hernán Crespo,Centre-Forward,25,Parma,Serie A,Lazio,Serie A,2000-2001,NaN,56810000
2,Marc Overmars,Left Winger,27,Arsenal,Premier League,FC Barcelona,LaLiga,2000-2001,NaN,40000000
3,Gabriel Batistuta,Centre-Forward,31,Fiorentina,Serie A,AS Roma,Serie A,2000-2001,NaN,36150000
4,Nicolas Anelka,Centre-Forward,21,Real Madrid,LaLiga,Paris SG,Ligue 1,2000-2001,NaN,34500000


In [3]:
transfers_df = transfers_df.rename(columns={"Name": "name", "Position": "position", "Age": "age", "Team_from": "team_from",
                             "League_to": "league_to","League_from": "league_from", "Team_to":"team_to", "Season":"season",
                             "Market_value":"market_value", "Transfer_fee":"transfer_fee"})
transfers_df['market_value'] = transfers_df['market_value'] .fillna(0)
# transfers_df.set_index("name", inplace=True)
transfers_df.head()

,name,position,age,team_from,league_from,team_to,league_to,season,market_value,transfer_fee
0,Luís Figo,Right Winger,27,FC Barcelona,LaLiga,Real Madrid,LaLiga,2000-2001,0.0,60000000
1,Hernán Crespo,Centre-Forward,25,Parma,Serie A,Lazio,Serie A,2000-2001,0.0,56810000
2,Marc Overmars,Left Winger,27,Arsenal,Premier League,FC Barcelona,LaLiga,2000-2001,0.0,40000000
3,Gabriel Batistuta,Centre-Forward,31,Fiorentina,Serie A,AS Roma,Serie A,2000-2001,0.0,36150000
4,Nicolas Anelka,Centre-Forward,21,Real Madrid,LaLiga,Paris SG,Ligue 1,2000-2001,0.0,34500000


In [4]:
locations_csv = "../Resources/locations.csv"
locations_df = pd.read_csv(locations_csv)
locations_df = locations_df.rename(columns={"Country":"country","Code": "country_code"})
# locations_df = locations_df.drop(columns=['Unnamed: 4', 'Unnamed: 5'])
locations_df = locations_df.dropna()
locations_df.set_index("country", inplace=True)
locations_df.head()

,country_code,latitude,longitude
country,,,
Afghanistan,AFG,33.939110,67.709953
Albania,ALB,41.153332,20.168331
Algeria,DZA,28.033886,1.659626
American Samoa,ASM,-14.270972,-170.132217
Andorra,AND,42.546245,1.601554


In [5]:
leagues_csv = "../Resources/league_countries.csv"
leagues_df = pd.read_csv(leagues_csv)
leagues_df = leagues_df.rename(columns={"League Name": 'league_name', "Country":"country"})
leagues_df.set_index("league_name", inplace=True)
leagues_df.head()

,country
league_name,
LaLiga,Spain
Serie A,Italy
Premier League,England
Ligue 1,France
LaLiga2,Spain


In [6]:
connection_string = "postgres:taunus48@localhost:5432/football_db"
engine = create_engine(f'postgresql://{connection_string}')

In [7]:
engine.table_names()

[]

In [8]:
locations_df.to_sql(name='locations', con=engine, if_exists='append', index=True)

In [9]:
leagues_df.to_sql(name='league_countries', con=engine, if_exists='append', index=True)


In [10]:
transfers_df.to_sql(name='transfers', con=engine, if_exists='append', index=True)

In [27]:
from flask import Flask, jsonify

In [31]:
# def from_data():
#     from_info = pd.read_sql_query("SELECT *\
#     FROM locations l\
#     JOIN league_countries c ON c.country = l.country\
#     JOIN transfers t ON t.league_from = league_name\
#     WHERE season='2010-2011'", con=engine).head()
#     return jsonify(from_info)


In [22]:
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [23]:
connection_string = "postgres:taunus48@localhost:5432/football_db"
engine = create_engine(f'postgresql://{connection_string}')

In [24]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

In [32]:
Base.classes.keys()

[]

In [33]:
engine.execute("""SELECT *\
    FROM locations l\
    JOIN league_countries c ON c.country = l.country\
    JOIN transfers t ON t.league_from = league_name\
    WHERE season='2010-2011'""")
    

ProgrammingError: (psycopg2.errors.SyntaxError) syntax error at or near "FROM"
LINE 1: SELECT *#     FROM locations l#     JOIN league_countries c ...
                      ^

[SQL: SELECT *#     FROM locations l#     JOIN league_countries c ON c.country = l.country#     JOIN transfers t ON t.league_from = league_name#     WHERE season='2010-2011']
(Background on this error at: http://sqlalche.me/e/13/f405)